In [ ]:
# === Environment Setup ===
import os, sys, math, time, random, json, textwrap, warnings
import numpy as np, pandas as pd, matplotlib.pyplot as plt
from numba import njit
from IPython.display import display, Markdown
# Load the line_profiler extension
%load_ext line_profiler

# --- Configuration ---
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams.update({'figure.dpi': 130, 'font.size': 12, 'axes.titlesize': 'x-large',
    'axes.labelsize': 'large', 'xtick.labelsize': 'medium', 'ytick.labelsize': 'medium'})

# --- Utility Functions ---
def note(msg, **kwargs):
    display(Markdown(f"<div class='alert alert-info'>📝 {textwrap.fill(msg, width=100)}</div>"))
def sec(title):
    print(f"\n{100*'='}\n| {title.upper()} |\n{100*'='}")

note("Environment initialized.")

# Part 1: Foundations
## Chapter 1.22: Profiling and Performance Optimization

### Introduction: From Correctness to Speed

Once your code is correct and well-tested, the next frontier is often performance. In computational economics, models can take hours or even days to run. The ability to identify and eliminate performance **bottlenecks** is a critical skill. **Profiling** is the systematic process of measuring the resource usage of your code—how much time it spends on each line or in each function—to find these bottlenecks.

The cardinal rule of optimization, famously articulated by Donald Knuth, is: **"Premature optimization is the root of all evil."** Do not attempt to optimize code before you have: 
1. Ensured it is correct (via tests).
2. Profiled it to find the actual bottlenecks.

This notebook introduces the essential profiling tools available in the Python ecosystem.

### Case Study: A Buffer-Stock Savings Model Simulation
We will use a simple Monte Carlo simulation of a buffer-stock savings model as our test case. The model simulates the wealth of a household over time, which receives a stochastic income stream and saves to smooth consumption.

In [ ]:
def simulate_buffer_stock(n_sim=1000, T=500, r=0.01, sigma=0.1, rho=0.9, y_bar=1.0, c_bar=0.9):
    """Simulates wealth paths for a buffer-stock savings model."""
    wealth = np.zeros((T, n_sim))
    log_y = np.zeros((T, n_sim))
    
    for t in range(T - 1):
        log_y[t+1, :] = rho * log_y[t, :] + sigma * np.random.randn(n_sim)
        y = y_bar * np.exp(log_y[t+1, :])
        c = c_bar * wealth[t, :] + y
        wealth[t+1, :] = (1 + r) * (wealth[t, :] - c) + y
    return wealth

### 1. Quick Benchmarking with `%timeit`
The IPython 'magic' command `%timeit` is the simplest tool for performance measurement. It repeatedly runs a single line of code to get a precise estimate of its average execution time.

In [ ]:
sec("Benchmarking with %timeit")
%timeit simulate_buffer_stock()

### 2. Function-Level Profiling with `cProfile`
`%prun` (which uses the `cProfile` module) breaks down the total execution time by function. This is invaluable for identifying which functions are the most time-consuming.

In [ ]:
sec("Function-Level Profiling with %prun")
%prun simulate_buffer_stock()

### 3. Line-by-Line Profiling with `line_profiler`
The most granular tool is `line_profiler`. It tells you exactly how much time was spent on *each line* of a function, allowing you to pinpoint the exact source of a bottleneck.

In [ ]:
sec("Line-by-Line Profiling with %lprun")
# The -f flag specifies which function to profile
%lprun -f simulate_buffer_stock simulate_buffer_stock()

### 4. Optimization with Numba
The profiling results clearly show that the bottleneck is the `for` loop. A common and powerful way to accelerate such loops in numerical Python is to use **Numba**, a just-in-time (JIT) compiler. By simply adding the `@njit` decorator, Numba will compile the function to highly efficient machine code.

In [ ]:
@njit
def simulate_buffer_stock_numba(n_sim=1000, T=500, r=0.01, sigma=0.1, rho=0.9, y_bar=1.0, c_bar=0.9):
    """Numba-optimized version of the simulation."""
    wealth = np.zeros((T, n_sim))
    log_y = np.zeros((T, n_sim))
    
    for t in range(T - 1):
        log_y[t+1, :] = rho * log_y[t, :] + sigma * np.random.randn(n_sim)
        y = y_bar * np.exp(log_y[t+1, :])
        c = c_bar * wealth[t, :] + y
        wealth[t+1, :] = (1 + r) * (wealth[t, :] - c) + y
    return wealth

sec("Benchmarking the Numba-Optimized Version")
# Run once to compile
_ = simulate_buffer_stock_numba()
%timeit simulate_buffer_stock_numba()
note("The Numba version is typically orders of magnitude faster, demonstrating the power of JIT compilation for numerical loops.")